# Phân tích dữ liệu bất động sản


Nguồn dữ liệu: Dữ liệu được thu thập từ website Batdongsan.com.vn, chuyên mục Bán nhà riêng.


In [1]:
import pandas as pd
import numpy as np
import re
from unidecode import unidecode
import warnings
warnings.filterwarnings('ignore')
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

Đọc dữ liệu từ file csv


In [2]:
df = pd.read_csv('../data/raw/data.csv', encoding='utf-8')

# Drop cột 'Tiêu đề' và 'URL'
df = df.drop(columns=['Tiêu đề', 'URL', 'Ngày đăng'])

print("5 bản ghi đầu tiên")
df.head() 

5 bản ghi đầu tiên


,Địa điểm,Mức giá,Diện tích,Số phòng ngủ,"Số phòng tắm, vệ sinh",Số tầng,Hướng nhà,Hướng ban công,Đường vào,Mặt tiền,Pháp lý,Nội thất
0,"Hoàng Mai, Hà Nội",20 tỷ,133 m²,NaN,NaN,NaN,Đông - Nam,NaN,"1,8 m","8,6 m",Sổ đỏ/ Sổ hồng,NaN
1,"Phú Nhuận, Hồ Chí Minh","2,45 tỷ/m²",56 m²,5 phòng,4 phòng,4 tầng,NaN,NaN,6 m,4 m,Sổ đỏ/ Sổ hồng,#NAME?
2,"Ngũ Hành Sơn, Đà Nẵng",25 tỷ,110 m²,9 phòng,10 phòng,6 tầng,Tây - Nam,Tây - Nam,NaN,NaN,Sổ đỏ/ Sổ hồng,Đầy đủ
3,"Quận 1, Hồ Chí Minh","2,39 tỷ",61 m²,4 phòng,5 phòng,3 tầng,NaN,NaN,5 m,4 m,Sổ đỏ/ Sổ hồng.,Đầy đủ.
4,"Bình Thạnh, Hồ Chí Minh","2,59 tỷ","49,7 m²",4 phòng,5 phòng,3 tầng,NaN,NaN,6 m,NaN,Sổ đỏ/ Sổ hồng,Cơ bản


## I) Kiểm Tra Tổng Quan


### a) Thống kê số lượng giá trị thiếu trên mỗi bản ghi


In [3]:
missing_counts = df.isna().sum(axis=1)
missing_summary = missing_counts.value_counts().sort_index()
print(missing_summary)


0     4002
1     3014
2    11784
3     9378
4     7514
5     5154
6     3570
7     2358
8     1545
9      914
Name: count, dtype: int64


#### Nhận xét

- Số lượng bản ghi thiếu trên 3 giá trị chiếm tỷ lệ khá lớn, làm giảm độ tin cậy và chất lượng dữ liệu phân tích.

- Để đảm bảo kết quả phân tích chính xác và giảm thiểu nhiễu, ta nên loại bỏ các bản ghi này.

- Sau khi loại bỏ, dataset còn lại khoảng **25,000** bản ghi, đủ lớn để thực hiện các phân tích sâu và xây dựng mô hình một cách hiệu quả.


### b) Thống kê số lượng thiếu của các cột


In [4]:
def analyze_columns(df):
    """Phân tích giá trị thiếu và kiểu dữ liệu của các cột"""

    # Tính tỉ lệ thiếu của mỗi cột
    missing_stats = pd.DataFrame({
        'Số giá trị thiếu': df.isnull().sum(),
        'Tỉ lệ thiếu (%)': (df.isnull().sum() / len(df) * 100).round(2)
    })

    # Sắp xếp theo tỉ lệ thiếu giảm dần
    missing_stats = missing_stats.sort_values('Tỉ lệ thiếu (%)', ascending=False)

    print("\nKết quả phân tích cột:")
    print(missing_stats)

    return missing_stats

original_stats = analyze_columns(df)


Kết quả phân tích cột:
                       Số giá trị thiếu  Tỉ lệ thiếu (%)
Hướng ban công                    43031            87.40
Hướng nhà                         38805            78.82
Đường vào                         20316            41.27
Nội thất                          18751            38.09
Mặt tiền                          16621            33.76
Số phòng tắm, vệ sinh             12062            24.50
Số phòng ngủ                       8736            17.74
Số tầng                            6709            13.63
Pháp lý                            4023             8.17
Diện tích                             0             0.00
Địa điểm                              0             0.00
Mức giá                               0             0.00


#### Nhận xét:

- Một số cột có **tỉ lệ thiếu rất cao (> 70%)** như `Hướng ban công`, `Hướng nhà` → nên loại bỏ.
- Các cột có **tỉ lệ thiếu trung bình (30–50%)** như `Đường vào`, `Nội thất`, `Mặt tiền` → có thể giữ lại nếu xử lý hợp lý.


### c) Kiểm Tra Kiểu Dữ Liệu


In [5]:
df.dtypes

Địa điểm                 object
Mức giá                  object
Diện tích                object
Số phòng ngủ             object
Số phòng tắm, vệ sinh    object
Số tầng                  object
Hướng nhà                object
Hướng ban công           object
Đường vào                object
Mặt tiền                 object
Pháp lý                  object
Nội thất                 object
dtype: object

#### Nhận xét:

- Tất cả các cột trong DataFrame đều có kiểu dữ liệu `object`, mặc dù một số cột thực tế chứa dữ liệu số hoặc văn bản.
- Cần chuyển các cột như `Mức giá`, `Diện tích`, `Số phòng ngủ`, `Số phòng tắm, vệ sinh`, `Số tầng`, `Mặt tiền` thành kiểu `float` hoặc `int` để có thể thực hiện các phép toán và phân tích tiếp theo.
- Các cột như `Địa điểm`, `Hướng nhà`, `Hướng ban công`, `Đường vào`, `Pháp lý`, `Nội thất` có thể giữ kiểu `object`. Tuy nhiên, cần phải chuẩn hóa và kiểm tra dữ liệu để loại bỏ các lỗi chính tả hoặc các giá trị không hợp lệ, giúp dữ liệu trở nên đồng nhất hơn.


## II) Phân tích đặc trưng số (Numerical)


Các đặc trưng số trong dữ liệu bao gồm: **Mức giá**, **Diện tích**, **Số phòng ngủ**, **Số phòng tắm, vệ sinh**, **Số tầng**, **Đường vào**, và **Mặt tiền**.

- Các đặc trưng không có đơn vị (như số tầng, số phòng) → không cần kiểm tra đơn vị.
- Các đặc trưng có đơn vị (giá, diện tích, mặt tiền, đừng vào) → cần kiểm tra kỹ và chuẩn hóa.được chuẩn hóa để đảm bảo tính nhất quán.

Trong bước này, chúng ta sẽ tập trung vào kiểm tra và chuẩn hóa các đơn vị.


### 1) Mức giá


In [6]:
unit_counts = df['Mức giá'] \
    .apply(lambda x: re.sub(r'[0-9.,]+', '', str(x))) \
    .str.strip() \
    .value_counts()

# In kết quả
print("Số lượng từng đơn vị trong cột 'Mực giá':")
print(unit_counts)

Số lượng từng đơn vị trong cột 'Mực giá':
Mức giá
tỷ             44332
Thỏa thuận      4424
triệu/m²         277
triệu            194
nghìn              3
tỷ/m²              2
triệu/tháng        1
Name: count, dtype: int64


#### Nhận xét:

- **`tỷ`** là đơn vị phổ biến nhất.
- **`triệu`** cũng là đơn vị phù hợp, có thể quy đổi về `tỷ`.
- **`triệu/m²`** là đơn vị theo giá trên diện tích – có thể chuyển đổi sang tổng giá trị bằng cách **nhân với diện tích**.
- Các đơn vị như `Thỏa thuận`, `nghìn`, `tỷ/m²`, `triệu/tháng`:
  - Không cung cấp thông tin cụ thể hoặc không phù hợp với mục tiêu phân tích giá tổng → **sẽ bị loại bỏ**.

#### ✅ Kế hoạch xử lý:

- **Loại bỏ** các bản ghi có đơn vị: `Thỏa thuận`, `nghìn`, `tỷ/m²`, `triệu/tháng`.
- **Tính toán lại giá** cho các bản ghi có đơn vị `triệu/m²`:  
  → `Giá (triệu) = Giá trị * Diện tích`
- **Chuyển đổi đơn vị** `triệu` → `tỷ` để đồng nhất (chia cho 1000).


### 2) Diện tích


In [7]:
unit_counts = df['Diện tích'] \
    .apply(lambda x: re.sub(r'[0-9.,]+', '', str(x))) \
    .str.strip() \
    .value_counts()

# In kết quả
print("Số lượng từng đơn vị trong cột 'Diện tích':")
print(unit_counts)

Số lượng từng đơn vị trong cột 'Diện tích':
Diện tích
m²    49233
Name: count, dtype: int64


#### Nhận xét:

- Tất cả bác bản ghi đều có đơn vị là **`m²`**


### 3) Đường vào


Ý nghĩa: Chiều rộng của con đường dẫn từ trục đường chính tới bất động sản.


In [8]:
unit_counts = df['Đường vào'] \
    .apply(lambda x: re.sub(r'[0-9.,]+', '', str(x))) \
    .str.strip() \
    .value_counts()

# In kết quả
print("Số lượng từng đơn vị trong cột 'Đường vào':")
print(unit_counts)

Số lượng từng đơn vị trong cột 'Đường vào':
Đường vào
m      28917
nan    20316
Name: count, dtype: int64


#### Nhận xét:

- Tất cả bác bản ghi đều có đơn vị là **`m`**


### 4) Mặt tiền


Ý nghĩa: Chiều dài mặt trước của ngôi nhà/đất tiếp giáp trực tiếp với đường.


In [9]:
unit_counts = df['Mặt tiền'] \
    .apply(lambda x: re.sub(r'[0-9.,]+', '', str(x))) \
    .str.strip() \
    .value_counts()

# In kết quả
print("Số lượng từng đơn vị trong cột 'Mặt tiền':")
print(unit_counts)

Số lượng từng đơn vị trong cột 'Mặt tiền':
Mặt tiền
m      32612
nan    16621
Name: count, dtype: int64


#### Nhận xét:

- Tất cả bác bản ghi đều có đơn vị là **`m`**


## III) Phân tích đặc trưng phân loại (Categorical)


Các đặc trưng phân loại trong dữ liệu bao gồm: **Địa điểm**, **Hướng nhà**, **Pháp lý**, **Nội thất**.


### 1) Địa điểm


In [10]:
df['Địa điểm'].head()

0          Hoàng Mai, Hà Nội
1     Phú Nhuận, Hồ Chí Minh
2      Ngũ Hành Sơn, Đà Nẵng
3        Quận 1, Hồ Chí Minh
4    Bình Thạnh, Hồ Chí Minh
Name: Địa điểm, dtype: object

#### Nhận xét:

Trong dữ liệu, địa điểm được ghi theo định dạng "quận/huyện, tỉnh/thành phố" và được phân tách bằng dấu phẩy.  
 → Có thể tách thành 2 cột riêng biệt: "Quận/Huyện" và "Tỉnh/Thành phố" để thuận tiện cho việc phân tích hoặc nhóm theo khu vực.

Để giá trị này có thể áp dụng, đề xuất crawl dữ liệu về mật độ dân cư của các quận huyện để tạo feature mới


### 2) Hướng nhà


Hướng nhà có ý nghĩa về mặt phong thủ, có thể ảnh hướng tới giá nhà.


In [11]:
df['Hướng nhà'].value_counts()


Hướng nhà
Đông - Nam    2164
Tây - Nam     1427
Đông - Bắc    1376
Tây - Bắc     1299
Nam           1236
Đông          1147
Tây            949
Bắc            830
Name: count, dtype: int64

#### Nhận xét:

Dữ liệu trong cột **Hướng nhà** khá sạch, không xuất hiện lỗi định dạng như một giá trị có nhiều cách viết khác nhau.
→ Có thể giữ nguyên hoặc thực hiện chuẩn hóa nhẹ (ví dụ: loại bỏ khoảng trắng dư, viết hoa đồng nhất) để phục vụ phân tích sau này.


### 3) Pháp lý


In [12]:
df['Pháp lý'].value_counts()


Pháp lý
Sổ đỏ/ Sổ hồng                                                                             40377
Sổ đỏ/ Sổ hồng.                                                                             1764
Sổ đỏ                                                                                        804
Sổ hồng                                                                                      606
Hợp đồng mua bán                                                                             198
                                                                                           ...  
Pháp lý: Thuế phí chủ lo.                                                                      1
Sổ hồng hoàn công nhà                                                                          1
Sổ đỏ sổ hồng chính chủ                                                                        1
Pháp lý: Sổ hồng riêng, hoàn công đầy đủ. Ngân hàng hỗ trợ vay trên 70% cho khách hàng.        1
Giấy viết tay         

#### Nhận xét:

Dữ liệu trong cột Pháp lý có nhiều giá trị khác nhau, bao gồm thông tin về sổ đỏ, sổ hồng, hợp đồng mua bán, và các thông tin không rõ ràng như "giấy viết tay" hoặc "thuế phí chủ lo".

#### ✅ Kế hoạch xử lý:

- Lọc từ khóa và chia thành 2 loại: Có và Không.
- Cần nghiên kíu kỹ từ khóa lọc để phân loại chính xác vào các nhóm


### 4) Nội thất


In [13]:
df['Nội thất'].value_counts()


Nội thất
Đầy đủ                                                                                                                      17772
Cơ bản                                                                                                                       7603
Đầy đủ.                                                                                                                      1265
Không nội thất                                                                                                               1080
Cơ bản.                                                                                                                       485
                                                                                                                            ...  
Tặng Nội thất cao Cấp, Trang Bị sẵn theo thiết kế                                                                               1
Đầy đủ gồm giường tủ bàn ghế sofa, tủ lạnh, bếp từ, hút mùi, điều hòa nóng lạnh, 

#### Nhận xét:

Dữ liệu trong cột **Nội thất** có sự đa dạng về cách diễn đạt nhưng phần lớn có thể được gom nhóm theo các loại chính.

Ba nhóm phổ biến là:

- **Đầy đủ**: chiếm phần lớn, với các cách ghi như “Đầy đủ”, “Đầy đủ.” hoặc mô tả chi tiết nội thất.
- **Cơ bản**: cũng xuất hiện nhiều, gồm cả “Cơ bản” và “Cơ bản.”.
- **Không nội thất**: là nhóm còn lại nhưng ít hơn.

Ngoài ra có nhiều mô tả chi tiết, dài dòng (vd: “Đầy đủ gồm giường tủ bàn ghế sofa...”) nhưng thực chất vẫn thuộc nhóm **Đầy đủ**.

#### ✅ Kế hoạch xử lý:

Chuẩn hóa lại dữ liệu bằng cách lọc từ khóa (ví dụ: “đầy đủ”, “cơ bản”, “không”) để phân nhóm thống nhất giúp phân tích dễ dàng hơn.


## IV) Kết luận


### 1. Vấn đề cần xử lý

1. **Giá trị thiếu**:

   - Các cột có tỷ lệ thiếu cao (>70%): Hướng ban công, Hướng nhà
   - Các cột có tỷ lệ thiếu trung bình (30-50%): Đường vào, Nội thất, Mặt tiền
   - Các cột có tỷ lệ thiếu thấp (<20%): Số phòng ngủ, Số phòng tắm, Số tầng

2. **Chuẩn hóa đơn vị**:

   - Mức giá: Cần chuẩn hóa về đơn vị tỷ đồng
   - Diện tích: Đã thống nhất đơn vị m²
   - Đường vào và Mặt tiền: Đã thống nhất đơn vị m

3. **Chuẩn hóa dữ liệu phân loại**:
   - Địa điểm: Cần tách thành Quận/Huyện và Tỉnh/Thành phố
   - Pháp lý: Cần chuẩn hóa thành các nhóm rõ ràng
   - Nội thất: Cần chuẩn hóa thành 3 nhóm chính

### 2. Kế hoạch xử lý

1. **Tiền xử lý (01_preprocess.py)**:

   - Loại bỏ các cột có tỷ lệ thiếu cao
   - Chuẩn hóa đơn vị đo lường
   - Tách và chuẩn hóa dữ liệu địa điểm
   - Chuẩn hóa các cột phân loại

2. **Phân tích phân phối (02_analyze_distribution.ipynb)**:
   - Phân tích ngoại lệ trong giá và diện tích
   - Phân tích tương quan giữa các biến
   - Phân tích theo khu vực địa lý
